In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uniprot
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [2]:
seqids, fastas = uniprot.read_fasta(r"C:\Users\marcu\Natural Language Processing\Final_Project\uniprot_sprot.fasta\uniprot_sprot.fasta")

In [3]:
homosapiens = [ (sp,data) for sp,data in fastas.items() if 'Homo sapiens' in data['description'] ]

In [4]:
def createDataTuples(sequences):
    dataTuples = []
    c = 1
    for sp,info in sequences:
        protein = "protein"+str(c)
        dataTuples.append( (protein,info['sequence']) )
        c+=1
    return dataTuples

In [5]:
dt = createDataTuples(homosapiens)

In [6]:
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm1b_t33_650M_UR50S")

Using cache found in C:\Users\marcu/.cache\torch\hub\facebookresearch_esm_main


In [7]:
# batch_converter = alphabet.get_batch_converter()
# model.eval()

# batch_labels, batch_strs, batch_tokens = batch_converter(dt)
# with torch.no_grad():
#     results = model(batch_tokens, repr_layers=[33], return_contacts=True)
# token_representations = results["representations"][33]